In [ ]:
import time
import multiprocessing

def count_down(x):
    while True:
        if x == 0:
            break
        
        print("카운트타운 ... %d" % x)
        x -= 1
        time.sleep(1)
        
if __name__ == '__main__':
    p = multiprocessing.Process(target=count_down, args=(5, ))
    print(p)
    p.start()
    p.join()

In [ ]:
import threading
import os

# 제어권이 넘어가는 걸 확인할수 있다.

def info():
    print('모듈명: ', __name__)
    print('프로세스 id: ', os.getpid())
    print('스레드 id: ', threading.get_ident())
    print('이런식으로 많이 집어넣으면 Context Switching도 확인할 수 있음')

def print_hello():
    print('안녕')
    info()
    time.sleep(1)
    print('잠자다 일어난다\n')
    
if __name__ == '__main__':
    # 현재 시간 기록
    start = time.perf_counter()
    threads = []
    
    # 스레드 5개 생성
    # 실제 컴퓨터 성능에 따라서 이 부분에 많이 혼성될 수도 있고 덜 혼성될 수도 있음
    # 컴퓨터가 압도적으로 좋거나 압도적으로 후지면 잘 혼성되지 않는다.
    for _ in range(5):
        t = threading.Thread(target = print_hello)
        t.start()
        threads.append(t)
    
    # 스레드 구동
    for thread in threads:
        thread.join()
    
    # 현재 시간 기록(현재 시간 - 이전 시간) -> 걸린 시간
    finish = time.perf_counter()
    
    print('종료: ', str(round(finish - start, 2)) + '초')

In [ ]:
import threading

x = 0

def increment_global():
    global x
    x += 1

def thread_processing():
    for _ in range(1000000):
        increment_global()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))
    
# 데이터 무결성 -> 200만이 나와야하는데 안나온다.(결과값이 기대값과 다름) -> 데이터의 무결성을 보장할수 없다.

In [ ]:
import threading

# java에서는 async, await와 같음
# lock을 많이쓰면 속도가 느려진다 -> lock을 적절하게 써야함
lock = threading.Lock()
x = 0

def increment_global():
    global x
    x += 1

def thread_processing():
    for _ in range(1000000):
        lock.acquire()
        increment_global()
        lock.release()
        # 여기서 lock이 100만번 걸리게 됨 -> 느려지는 이유
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
start = time.perf_counter()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))
    
finish = time.perf_counter()
print('종료: ', str(round(finish - start, 2)) + '초')

In [ ]:
import threading

# java에서는 async, await와 같음
# lock을 많이쓰면 속도가 느려진다 -> lock을 적절하게 써야함
lock = threading.Lock()
x = 0

def increment_global():
    global x
    x += 1

def thread_processing():
    # lock을 100만번 반복에서 제외시킴 -> 빨라짐
    lock.acquire()
    for _ in range(1000000):
        increment_global()
    
    lock.release()
        
def thread_main():
    global x
    x = 0
    
    t1 = threading.Thread(target = thread_processing)
    t2 = threading.Thread(target = thread_processing)
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()
    
start = time.perf_counter()
    
for i in range(10):
    thread_main()
    print("{0} 번째 반복 이후 x = {1}".format(i, x))
    
finish = time.perf_counter()
print('종료: ', str(round(finish - start, 2)) + '초')

In [ ]:
# MySQL install
!pip install pymysql

In [1]:
MYSQL_USER_DATA_SAVED_FILE = "mysql/userinfo"

In [2]:
import pickle

mysql_user_info = dict({
   'user_id': 'eddi',
   'password': 'eddi@123'
})

f = open(MYSQL_USER_DATA_SAVED_FILE, 'wb')
pickle.dump(mysql_user_info, f)
f.close()

In [3]:
mysql_user_info

{'user_id': 'eddi', 'password': 'eddi@123'}

In [5]:
del mysql_user_info
#mysql_user_info

In [6]:
import pickle

f = open(MYSQL_USER_DATA_SAVED_FILE, 'rb')
mysql_user_info = pickle.load(f)
f.close()

mysql_user_info

{'user_id': 'eddi', 'password': 'eddi@123'}

In [8]:
import pymysql

# 스키마 생성 - pydb
db = pymysql.connect(
    host = '127.0.0.1',
    port = 3306,
    user = mysql_user_info['user_id'],
    passwd = mysql_user_info['password'],
    db = 'pydb'
)

print(db)

In [9]:
cursor = db.cursor()

sql = """
    create table pydb_test(
        id int unsigned not null auto_increment,
        name varchar(20) not null,
        price int not null,
        primary key(id)
    )
"""

cursor.execute(sql)
db.commit()
db.close()
print('테이블 생성 성공')

테이블 생성 성공
